In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split

In [2]:
path='./Resources/job.csv'
df=pd.read_csv(path)
df.head(3)

,job_id,title,location,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York","We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland","90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0


In [3]:
df.head()

,job_id,title,location,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York","We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland","90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [4]:
df_new=df.fillna(" ")

In [5]:
y=df_new['fraudulent'].astype(float)
X=df_new['description']

In [6]:

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(X_train)

<tf.Tensor: shape=(13410, 20), dtype=float32, numpy=
array([[ 5.8829026 , -0.18041724,  4.5786443 , ...,  1.0470891 ,
         1.4603465 , -2.4322183 ],
       [ 3.388894  , -1.4281816 ,  4.459769  , ..., -0.5110417 ,
         1.148909  , -1.7515355 ],
       [ 0.24683896, -1.2276443 ,  1.3424275 , ..., -0.505376  ,
         0.8152147 , -0.43984973],
       ...,
       [-1.3538884 , -0.39882982, -1.1260065 , ..., -0.05121366,
         1.9099666 ,  0.6364785 ],
       [-0.159909  , -0.7711938 ,  1.2216191 , ..., -1.2455738 ,
        -1.0659382 , -0.38006982],
       [ 0.5137812 , -1.7106069 ,  3.1433709 , ...,  0.19052011,
         0.8236558 , -0.35969323]], dtype=float32)>

In [8]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(60, activation='relu'))
model.add(tf.keras.layers.Dense(25, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                389380    
_________________________________________________________________
dense (Dense)                (None, 60)                1260      
_________________________________________________________________
dense_1 (Dense)              (None, 25)                1525      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 26        
Total params: 392,191
Trainable params: 392,191
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test,y_test))

Epoch 1/20
420/420 [==============================] - 2s 4ms/step - loss: 0.1773 - accuracy: 0.9522 - val_loss: 0.1449 - val_accuracy: 0.9530
Epoch 2/20
420/420 [==============================] - 2s 4ms/step - loss: 0.1178 - accuracy: 0.9605 - val_loss: 0.1063 - val_accuracy: 0.9660
Epoch 3/20
420/420 [==============================] - 2s 4ms/step - loss: 0.0788 - accuracy: 0.9722 - val_loss: 0.0934 - val_accuracy: 0.9711
Epoch 4/20
420/420 [==============================] - 2s 4ms/step - loss: 0.0486 - accuracy: 0.9828 - val_loss: 0.0907 - val_accuracy: 0.9745
Epoch 5/20
420/420 [==============================] - 2s 4ms/step - loss: 0.0316 - accuracy: 0.9896 - val_loss: 0.1060 - val_accuracy: 0.9749
Epoch 6/20
420/420 [==============================] - 2s 4ms/step - loss: 0.0187 - accuracy: 0.9931 - val_loss: 0.1075 - val_accuracy: 0.9776
Epoch 7/20
420/420 [==============================] - 2s 4ms/step - loss: 0.0098 - accuracy: 0.9968 - val_loss: 0.1341 - val_accuracy: 0.9796
Epoch 

In [11]:
print(model.evaluate(X_test,y_test))

140/140 [==============================] - 0s 2ms/step - loss: 0.1840 - accuracy: 0.9743
[0.1839970499277115, 0.9742729067802429]


In [ ]:
model.save("savemodel")

In [12]:
from sklearn.metrics import classification_report
prediction = model.predict(X_test)
prediction = np.array([1 if row>=0.5 else 0 for row in prediction])
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      4253
         1.0       0.76      0.69      0.72       217

    accuracy                           0.97      4470
   macro avg       0.87      0.84      0.85      4470
weighted avg       0.97      0.97      0.97      4470



In [16]:
import mlflow
import mlflow.tensorflow

mlflow.tensorflow.autolog()

In [ ]:
loaded_model = tf.keras.models.load_model('savemodel')

In [ ]:
loaded_model.summary()

In [17]:
mlflow.keras.save_model(model, "keras_model")

In [18]:
loaded_keras_model = mlflow.keras.load_model("keras_model", custom_objects={'KerasLayer':hub.KerasLayer})

In [ ]:
X_test[:2].to_json(orient="split")

In [20]:
tf.compat.v1.keras.experimental.export_saved_model(model, 'model.h5')

Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.


Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


...

INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: model.h5/saved_model.pb


INFO:tensorflow:SavedModel written to: model.h5/saved_model.pb


In [21]:
loaded_model = tf.compat.v1.keras.experimental.load_from_saved_model("model.h5", custom_objects={'KerasLayer':hub.KerasLayer})

In [13]:
X_test.head()

16418    (We have more than 1500+ Job openings in our w...
11698    We are seeking for an analytical and highly en...
5915     PINT Inc. is seeking a professional administra...
5009     The Customer Service Associate will be based i...
2190     Hi, We have the below urgent position is open ...
Name: description, dtype: object

In [27]:
loaded_model.predict_classes(["this is a job-posting"])

array([[1]], dtype=int32)